<a href="https://colab.research.google.com/github/makhmudovamunira/pnevmoniya/blob/main/Pnevmoniya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import fastai
import torch

In [ ]:
!wget https://github.com/makhmudovamunira/pnevmoniya/blob/main/kaggle.json

##Kaggle API Tokenini to'g'ri joyga yuklash

In [ ]:
import shutil
import os

if os.path.exists("/root/.kaggle/kaggle.json"):
    os.remove("/root/.kaggle/kaggle.json")

os.makedirs("/root/.kaggle", exist_ok=True)

shutil.move("/content/kaggle.json", "/root/.kaggle/")

# Kaggle API ruxsatlarini o'rnatamiz
os.chmod("/root/.kaggle/kaggle.json", 600)


print("Kaggle API token muvaffaqiyatli o‘rnatildi.")

## Datasetni yuklash

In [ ]:
!kaggle competitions download -c pnevmoniya

In [ ]:
path='/kaggle/input/pnevmoniya/train'
print(os.listdir(path))

In [ ]:
!unzip /content/pnevmoniya.zip

In [ ]:
path='/content/train'
print(os.listdir(path))

In [ ]:
from fastai.vision.all import *
from ipywidgets import widgets

#datablock yaratamiz
lungs=DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=lambda x: get_image_files(x),
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224)
)

#dataloader qilamiz
dls=lungs.dataloaders(path)

#datasetni tekshiramiz bunda max n ta image chiqarib beradi
#dls.show_batch(max_n=28, nrows=4)

#train qilamiz
learn=cnn_learner(dls, resnet50, metrics=accuracy)
#learn.model.to('cuda')  #shart emas fastai avtomatik gpu ni ishga tushiradi
learn.fine_tune(10)

In [ ]:
#tekshiramiz
interp=ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
#eng katta qilingan xatosini tekshirishini so'raymiz
interp.plot_top_losses(4,nrows=1)

##Test qilib csv filega joylaymiz

In [ ]:
from pathlib import Path
test_path = Path("/content/test")  # Test papka manzili
test_images = list(test_path.iterdir())  # Barcha fayllarni ro‘yxatga olish

In [ ]:
predictions=[]
for img_path in test_images:
  img=PILImage.create(img_path)
  pred, pred_id, probs=learn.predict(img)
  if pred=='PNEUMONIA':
    label=1
  else:
    label=0
  predictions.append((img_path.name, label))

df=pd.DataFrame(predictions, columns=['id', 'labels'])
df.to_csv('PnevmoniyaPred.csv', index=False)
print("Saqlandi")